In [ ]:
def fit_rf_model_random_search(df_kl, all_labels, KL_type, threshold, n_splits=5, n_iters=100):
    if os.path.isfile(f'{PATH_MODELS}/{threshold}_RF_{KL_type}.full_data.joblib') == False:
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=243)
        data_kltype = {}

        param_grid = {
            'bootstrap': Categorical([True, False]),
            'max_depth': Integer(10, 100),
            'max_features': Categorical(['auto', 'sqrt']),
            'min_samples_leaf': Integer(1, 4),
            'min_samples_split': Integer(2, 10),
            'n_estimators': Integer(200, 800)
        }

        for n, (train_index, test_index) in enumerate(skf.split(df_kl, all_labels)):
            X_train, X_test = df_kl.iloc[train_index], df_kl.iloc[test_index]
            y_train, y_test = pd.Series(all_labels).iloc[train_index], pd.Series(all_labels).iloc[test_index]

            if n == 0:
                rf = RandomForestClassifier(random_state=42)
                bayes_search = BayesSearchCV(rf, param_grid, n_iter=n_iters, cv=4, n_jobs=-1)
                bayes_search.fit(X_train, y_train)
                best_params = bayes_search.best_params_
                best_model = bayes_search.best_estimator_
            else:
                best_model = RandomForestClassifier(random_state=42, **best_params)
                best_model.fit(X_train, y_train)

            predictions = best_model.predict(X_test)
            data_kltype[n] = {
                "best_parameters": best_params,
                "model": best_model,
                "test_data": (y_test, predictions),
                "test_&_model_predictions": (X_test, y_test),
                "iteration": n
            }

        joblib.dump(data_kltype, f'{PATH_MODELS}/{threshold}_RF_{KL_type}.full_data.joblib')
    else :
        pass
